In [74]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("prasad22/healthcare-dataset")

print("Path to dataset files:", path)

import os

# Check contents of the directory
print(os.listdir(path))

Path to dataset files: /root/.cache/kagglehub/datasets/prasad22/healthcare-dataset/versions/2
['healthcare_dataset.csv']


In [75]:
file_path = os.path.join(path, "healthcare_dataset.csv")

import pandas as pd
df = pd.read_csv(file_path)

print(df.head())

            Name  Age  Gender Blood Type Medical Condition Date of Admission  \
0  Bobby JacksOn   30    Male         B-            Cancer        2024-01-31   
1   LesLie TErRy   62    Male         A+           Obesity        2019-08-20   
2    DaNnY sMitH   76  Female         A-           Obesity        2022-09-22   
3   andrEw waTtS   28  Female         O+          Diabetes        2020-11-18   
4  adrIENNE bEll   43  Female        AB+            Cancer        2022-09-19   

             Doctor                    Hospital Insurance Provider  \
0     Matthew Smith             Sons and Miller         Blue Cross   
1   Samantha Davies                     Kim Inc           Medicare   
2  Tiffany Mitchell                    Cook PLC              Aetna   
3       Kevin Wells  Hernandez Rogers and Vang,           Medicare   
4    Kathleen Hanna                 White-White              Aetna   

   Billing Amount  Room Number Admission Type Discharge Date   Medication  \
0    18856.281306    

In [76]:
df.dtypes

,0
Name,object
Age,int64
Gender,object
Blood Type,object
Medical Condition,object
Date of Admission,object
Doctor,object
Hospital,object
Insurance Provider,object
Billing Amount,float64


In [77]:

# Convert columns to datetime objects
df['Date of Admission'] = pd.to_datetime(df['Date of Admission'], format='%Y-%m-%d')
df['Discharge Date'] = pd.to_datetime(df['Discharge Date'], format='%Y-%m-%d')

# Calculate the length of stay in days and add as a new column
df['Length of Stay'] = (df['Discharge Date'] - df['Date of Admission']).dt.days

# Display the updated DataFrame
#print(df.head())


In [78]:
df.drop(columns=['Name','Date of Admission','Discharge Date','Room Number'],inplace=True)
df.drop(columns=['Insurance Provider','Billing Amount'],inplace=True)
df.drop(columns=['Doctor','Hospital'],inplace=True)

In [79]:
df.columns

Index(['Age', 'Gender', 'Blood Type', 'Medical Condition', 'Admission Type',
       'Medication', 'Test Results', 'Length of Stay'],
      dtype='object')

In [80]:
import numpy as np

df.isnull().sum()

,0
Age,0
Gender,0
Blood Type,0
Medical Condition,0
Admission Type,0
Medication,0
Test Results,0
Length of Stay,0


In [81]:
obj_col=[]
num_col=[]
for i in df.columns:
  if df[i].dtype=='object':
    obj_col.append(i)
  else:
    num_col.append(i)

print("Object columns: ",obj_col)
print("Numerical columns: ",num_col)

Object columns:  ['Gender', 'Blood Type', 'Medical Condition', 'Admission Type', 'Medication', 'Test Results']
Numerical columns:  ['Age', 'Length of Stay']


In [82]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for i in df.columns:
  if df[i].dtype=='object':
    df[i]=le.fit_transform(df[i])

In [83]:
for u in df['Test Results'].unique():
  print(u, list(df['Test Results']).count(u))

2 18517
1 18356
0 18627


In [84]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Test Results']),df['Test Results'],test_size=0.35,random_state=42,stratify=df['Test Results'])

In [85]:
from sklearn.preprocessing import StandardScaler

ss=StandardScaler()
X_train[num_col]=ss.fit_transform(X_train[num_col])
X_test[num_col]=ss.transform(X_test[num_col])

In [87]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.3312226512226512


In [88]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm=SVC(kernel='linear')
svm.fit(X_train,y_train)
y_pred=svm.predict(X_test)
print(accuracy_score(y_test,y_pred))


0.3349292149292149


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm=SVC(kernel='rbf')
svm.fit(X_train,y_train)
y_pred=svm.predict(X_test)
print(accuracy_score(y_test,y_pred))
